In [1]:
import sys
sys.path.append("/Users/mocquin/MYLIB10/MODULES/noise3D/")

In [2]:
import numpy as np

import noise3d

# We'll be creating 3D sequences with T=100 frames, V=100 lines, and H=100 columns.
T = 100
V = 100
H = 100
# but you can define any shape

# Create some seq to apply operators on
seq = noise3d.genseq.genseq_3dnoise_seq(T, V, H, [1 for i in range(7)], [0 for i in range(7)])

# Basic noise variances

In [3]:
# Getting the individual noise variances 
print(noise3d.noise.var_nt(seq))
print(noise3d.noise.var_nv(seq))
print(noise3d.noise.var_nh(seq))
print(noise3d.noise.var_ntv(seq))
print(noise3d.noise.var_nth(seq))
print(noise3d.noise.var_nvh(seq))
print(noise3d.noise.var_ntvh(seq))
print()

# Wrapping all in a function, as well as total variance
noise_vars = noise3d.noise.get_all_3d_noise_var(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")



0.98556630379211
1.1008211421948055
0.8894923759569133
0.9801933479616582
0.9753709385404218
0.9692415912602789
0.9689348097082924

Var(t  ) : 0.99
Var(v  ) : 1.10
Var(h  ) : 0.89
Var(tv ) : 0.98
Var(th ) : 0.98
Var(vh ) : 0.97
Var(tvh) : 0.97
Var(tot) : 6.84


## Faster equivalent version

A faster equivalent version 

In [4]:
noise_vars = noise3d.noise.get_all_3d_noise_var_fast(seq, names=True)
for var, name in zip(noise_vars[:-1], noise_vars[-1]):
    print(f"Var({name:3}) : {var:.2f}")


Var(t  ) : 0.99
Var(v  ) : 1.10
Var(h  ) : 0.89
Var(tv ) : 0.98
Var(th ) : 0.98
Var(vh ) : 0.97
Var(tvh) : 0.94
Var(tot) : 6.84


In [5]:
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_noise_var(seq)

6.42 ms ± 741 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
43.6 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Matrix approach

A matrix unmixin approach is possible to compute the 7 noise variances.
The inputs of this approach are : 
 - a sequence
 - a mixin (or unmixin) matrix
 
The variances can be computed throu the helper, generic function \_get_all_3d_variance_from_matrix.  
The "quality" of the approach depends on the matrix coefficients.  
The classic approach variances can be computed throug the matrix approach using the matrix M_classic:

In [6]:
print(noise3d.noise.M_classic)

[[1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0]
 [1 0 1 0 1 0 0]
 [0 1 1 0 0 1 0]
 [1 1 1 1 1 1 1]]


In [7]:
sequence_approach = noise3d.noise.get_all_3d_noise_var(seq, names=True)
# using the generic function
matrix_approach_generic = noise3d.noise._get_all_3d_variance_from_matrix(seq, noise3d.noise.M_classic)

for var1, var2, name in zip(sequence_approach[:-1], matrix_approach_generic, sequence_approach[-1]):
    print(f"Variance {name:3} tol : {var1/var2-1:.5f}")

Variance t   tol : 0.00000
Variance v   tol : 0.00000
Variance h   tol : 0.00000
Variance tv  tol : 0.02153
Variance th  tol : 0.01940
Variance vh  tol : 0.02075
Variance tvh tol : -0.02895
Variance tot tol : 0.00000


A wrapped version is avalable.

In [8]:
matrix_approach = noise3d.noise.get_all_3d_classic_var_matrix(seq)
matrix_approach == matrix_approach_generic

True

Then, another matrix, called "corrected" matrix, which is provides unbiased variances estimations. Its values depends on the sequence size.


In [10]:
print(noise3d.noise.compute_M_corrected(T, V, H))

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e-02 1.0000000e-02
  0.0000000e+00 1.0000000e-04]
 [0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e-02 0.0000000e+00
  1.0000000e-02 1.0000000e-04]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 1.0000000e-02
  1.0000000e-02 1.0000000e-04]
 [9.9009901e-01 9.9009901e-01 0.0000000e+00 1.0000000e+00 9.9009901e-03
  9.9009901e-03 1.0000000e-02]
 [9.9009901e-01 0.0000000e+00 9.9009901e-01 9.9009901e-03 1.0000000e+00
  9.9009901e-03 1.0000000e-02]
 [0.0000000e+00 9.9009901e-01 9.9009901e-01 9.9009901e-03 9.9009901e-03
  1.0000000e+00 1.0000000e-02]
 [9.9000099e-01 9.9000099e-01 9.9000099e-01 9.9990100e-01 9.9990100e-01
  9.9990100e-01 1.0000000e+00]]


We can then compute variances more "accurately" with this matrix, using a wrapped function :

In [12]:
corrected = noise3d.noise.get_all_3d_corrected_var_matrix(seq)
print(corrected)

(0.9657154567401707, 1.081032826867643, 0.869753258948089, 0.9900093032486474, 0.9850894714148613, 0.9788362989371393, 0.9985930530418772, 6.869029669198428)


# Comparison of methods

In [18]:
# in terms of time
%timeit noise3d.noise.get_all_3d_noise_var(seq)
%timeit noise3d.noise.get_all_3d_noise_var_fast(seq)
%timeit noise3d.noise.get_all_3d_classic_var_matrix(seq)
%timeit noise3d.noise.get_all_3d_corrected_var_matrix(seq)

42.2 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.3 ms ± 233 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.2 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.25 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
# values
classic = noise3d.noise.get_all_3d_noise_var(seq, names=True)
names = classic[-1]
classic = classic[:-1]
classic_fast = noise3d.noise.get_all_3d_noise_var_fast(seq)
classic_matrix = noise3d.noise.get_all_3d_classic_var_matrix(seq)
corrected_matrix = noise3d.noise.get_all_3d_corrected_var_matrix(seq)

for cla, claf, clam, corm, name in zip(classic, classic_fast, classic_matrix, corrected_matrix, names):
    print(f"Variance {name}")
    print(cla)
    print(claf)
    print(clam)
    print(corm)
    print()

Variance t
0.98556630379211
0.9855663037921099
0.9855663037921099
0.9657154567401707

Variance v
1.1008211421948055
1.100821142194805
1.100821142194805
1.081032826867643

Variance h
0.8894923759569133
0.8894923759569136
0.8894923759569132
0.869753258948089

Variance tv
0.9801933479616582
0.9801933479616582
0.959536046516243
0.9900093032486474

Variance th
0.9753709385404218
0.9753709385404218
0.9568060011171649
0.9850894714148613

Variance vh
0.9692415912602789
0.9692415912602789
0.9495355168231332
0.9788362989371393

Variance tvh
0.9689348097082924
0.938889401541366
0.9978177148471845
0.9985930530418772

Variance tot
6.839575101247554
6.839575101247554
6.839575101247553
6.869029669198428

